# 2.9 Modular Linear Algebra

Readings went over some Lin Alg reiew including definitions for:
- Matrix
- determinant
- identity Matrix
- Vector multiplication
- Matrix multiplication

Not going over them here for sake of time, they are easily google-able

## 2.9.2 Exercise

Show $AI = IA = A$

> skip

## 2.9.3 Theorem of Multiplicity of Determinant

det(I) = 1

det(AB) = det(A)det(B)

## 2.9.6 Defn (Matrix Modular Congruency)

For

$$ A = \begin{bmatrix} a & b\\ c & d\end{bmatrix} $$
$$ B = \begin{bmatrix} a' & b'\\ c' & d'\end{bmatrix} $$

If

$$
a \equiv a' \text{  (mod n)}\\
b \equiv b' \text{  (mod n)}\\
c \equiv c' \text{  (mod n)}\\
d \equiv d' \text{  (mod n)}\\
$$

Then

$$ A \equiv B \text{  (mod n)} $$

## 2.9.7 Defn (Matrix Modular Invertibility)

A matrix A "is invertile mod n" if there exists a matrix X such that
$$ AX \equiv I \text{  (mod n)} $$

If so, then X is "an inverse of A mod n"
$$ X \equiv A^{-1} \text{  (mod n)} $$

## 2.9.8 Matrix Inversion Theorem

Suppose $ A = \begin{bmatrix} a & b\\ c & d \end{bmatrix}$ is a matrix.

Then $A$ is invertible IFF $det(A)$ is invertible mod n.

If $e \equiv det(A)^{-1}$ (mod n), then

$$ X = \begin{bmatrix}ed & -eb\\ -ec & ea\end{bmatrix} $$

X is an inverse of A mod n

### Proof (attempt)

Couldn't figure out a derivation so I settled for the proof

Given,
$$
\begin{align*}
det (A) &= ad - bc \\
e(det(A)) &\equiv 1 \text{  (mod n)}\\
\end{align*}
$$

Then,
$$
\begin{align*}
AX &= \begin{bmatrix} 
            a(ed)+b(-ec) & a(-eb)+b(ea)\\ 
            c(ed)+d(-ec) & c(-eb)+d(ea)
    \end{bmatrix}\\

   &= \begin{bmatrix} 
            e(ad-bc) & e(-ab+ba)\\ 
            e(cd-dc) & e(-cb+da)
    \end{bmatrix}\\

   &\equiv \begin{bmatrix} 
            1 & 0\\ 
            0 & 1
    \end{bmatrix} \text{  (mod n)} \\

   &\equiv I \text{  (mod n)} \\
\end{align*}
   
$$

# 2.10 Hill Cipher

1. Encoding into ASCII values
2. Find an key, A (nxn matrix), that's [invertible mod 26](#297-defn-matrix-modular-invertibility)
3. Encoded message into nx1 vectors and iterate
   - compute `Av mod 26` to encrypt each message vector

To decode, use inverse key (X) and iterate ciphertext vectors, applying `Xv mod 26`

## 2.10.1 Exercise

Practice encryption and decryption

In [1]:
# I GUESS I FINALLY HAVE TO FIGURE OUT EXTENDED GCD
n, a = 26, 17

def gcd(n, a):
    while (not n % a == 0):
        n, a = a, n % a
    
    return a

# n = qa + r
# n - qa = r
def ext_gcd(n, a):
    if a == 0:
        return 1, 0, n
    
    q = n//a
    r = n%a
    x, y, g = ext_gcd(a, r)
    return y, x - q*y, g

x, y, g = ext_gcd(n, a)
x*n+y*a == g

True

In [2]:
def det(matrix):
    if (not len(matrix) == len(matrix[0])):
        return "CANNOT FIND DET OF NON-SQUARE MATRIX"
    
    n = len(matrix) - 1
    return matrix[0][0]*matrix[n][n] - matrix[0][n]*matrix[n][0]

def coeff(C, mat):
    return [[C*c for c in r] for r in mat]

# [[a, b], [c, d]] * [[x], [y]]
# [[ax + by], [cx + dy]]
def vec(mat, v):
    if (not len(mat[0]) == len(v)):
        return "invalid mult"
    return (lambda arr: (arr[0], arr[1]))([sum(r[i]*v[i] for i in range(len(v)))%26 for r in mat])

# Enc/Dec goes through same process. Only diff is the key (A enc) (X dec)
def Hill(enc, mat):
    return ''.join([chr(c+0x41) for c in list(sum([vec(mat, v) for v in enc], ()))])

A = [[3, 2], [1, 7]]
e = ext_gcd(26, det(A))[1]
X = coeff(e, A)

message = [ord(c)-0x41 for c in 'GOTOLAKELERNA']
cipher = [ord(c)-0x41 for c in 'RNCQYVFRRLZI']

# Encoding Go To Lake Lerna
print(Hill([(message[2*i],message[2*i+1]) for i in range(len(message)//2)], A))

# Decoding RNCQYVFRRLZI
print(Hill([(cipher[2*i],cipher[2*i+1]) for i in range(len(cipher)//2)], X))

UAHNHLMMPNZE
PSCGGJTMXUNH


FUCK! 

KATIE LIU WAS RIGHT, I NEEDED TO INVERT THE MATRIX TOO BEFORE MULTIPLYINB BY THE INV_DET

SCREW MANUAL SCRIPTING I'M JUST GONNA USE NUMPY

In [3]:
import numpy as np
from numpy.linalg import det, inv

def mod_inv(mat):
    d = det(A)
    e = ext_gcd(26, round(d))[1]
    return e*d*inv(mat) % 26

# Testing: WORKED
# mod_inv(np.array([[3, 2], [1, 7]]))

# Encoding into proper form
def encode(msg):
    msg = [ord(c)-0x41 for c in msg]
    return [np.array([msg[2*i], msg[2*i+1]]).reshape(2, 1) for i in range(len(msg)//2)]
    

# Enc/Dec goes through same process. Only diff is the key (A enc) (X dec)
def Hill(msg, mat):
    return ''.join([chr(int(c) % 26 + 0x41) for c in (mat@encode(msg)).flatten()])

def demo():
    A = np.array([[3, 2], [1, 7]])
    X = mod_inv(A)
    msg = 'YOUHAVESAVEDUSALLZ'
    cipher = Hill(msg, A)

    # Testing with example given in slide
    Hill(cipher, X), Hill(msg, A)

# 2.10.2 Sagecell exercise
A = np.array([[3, -1], [2, 5]])
X = mod_inv(A)
Hill('GOTOLAKELERNAP', A), Hill('RNCQYVFRRLZI', X)

('EEREHWAODQMVLX', 'SLAYTHEHYDRA')

# Lecture Notes

## Polygraphic Ciphers

In particular, digraphic still substitutions, but we replace 2 letters at a time
- Hill
- Playfair

In [25]:
M = np.matrix(([4, 3], [1, 2]))
d = round(det(M))
e, _, g = ext_gcd(d, 26)
X = e*d*inv(M) % 26

g, e, d*inv(M), X, A@X % 26

(1,
 -5,
 matrix([[ 2., -3.],
         [-1.,  4.]]),
 matrix([[16., 15.],
         [ 5.,  6.]]),
 matrix([[17., 13.],
         [ 5.,  8.]]))

In [63]:
message = 'AREA'
encoded = encode(message)

message, M, "",  encoded , "", M@encoded, "", [(M@encoded % 26 + 0x41).flatten()]#, ''.join([chr(0x41 + c) for c in (M@encoded % 26).flatten()]) #, Hill(message, M)

('AREA',
 matrix([[4, 3],
         [1, 2]]),
 '',
 [array([[ 0],
         [17]]),
  array([[4],
         [0]])],
 '',
 matrix([[51, 34],
         [16,  4]]),
 '',
 [matrix([[90, 73, 81, 69]], dtype=int32)])

In [82]:
ciphertext = 'CRZX'

print(X)
print()

encoded = np.array(encode(ciphertext)) 
print(encoded)
print()
print(X @ encoded)
print()
print(X @ encoded % 26)

print()
print([(X @ encoded % 26).flatten()])

[[16. 15.]
 [ 5.  6.]]

[[[ 2]
  [17]]

 [[25]
  [23]]]

[[287. 112.]
 [745. 263.]]

[[ 1.  8.]
 [17.  3.]]

[matrix([[ 1.,  8., 17.,  3.]])]


### Playfair practice

Use FAJITAS as key to generate the table:

```
FAITS
BCDEG
HKLMN
OPQRU
VQXYZ
```

Encode "Little Fluffy" into pairs
```
LI TT LE FL UF FY
LI TX TL EF LU FF Y
LI TX TL EF LU FX FY
```

Encrypt it now
```
QD IY IM BT NQ IV TV
```
